#### I found a bugh while training a random model

For some reason the ntilde kept growing and this lead testing 1080 ntilde with 1080 remaining idx. 

This is exactly half of the " complete " dataset, having excluded 1000 test images.

Running this notebook should reproduce the bug.



In [1]:
import torch
torch.set_grad_enabled(False)
import sys
import time
import random
import csv
import numpy as np
import threading 
import scipy.io
from scipy.io import loadmat

import pickle

from tqdm import tqdm
from tqdm import notebook

import matplotlib
import matplotlib.pyplot as plt

import importlib

sys.path.append('../')
import utils

import copy
# Enable autoreload
%load_ext autoreload
%autoreload 2
importlib.reload(utils)

# Set seeds for reproducibility
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

TORCH_DTYPE = torch.float64 #NB: Basically all of the matrices in Spatial_GP have 1.e-7 added to the diagonal, to be changed if we want to use float64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
torch.set_default_dtype(TORCH_DTYPE)
torch.set_default_device(device)
print(f'Device is: {device}')


Using device: cuda:0 (from utils.py)
Using device: cuda:0 (from utils.py)
Device is: cuda:0


In [11]:
cellid       = 8         # Choose cell
ntrain_start = 50        # Number of first training data points

kernfun      = 'acosker' # Choose kernel function

nEstep       = 10         # Total number of E-steps iterations.
nFparamstep  = 10  
nMstep       = 10         # Total number of M-steps iterations. 
maxiter      = 10         # Iterations of the optimization algorithm comprising M and E steps

ntilde       = ntrain_start
n_test_lk    = 1000       # Number of test data points on which to compute log-likelihood, to compare 2 models without using the r2 score

# region _______ Import dataset and fix random indices _______
# Open the .pkl dataset file for reading in binary mode (rb)
with open('/home/idv-eqs8-pza/IDV_code/Variational_GP/spatial_GP/Data/data2_41mixed_tr28.pkl', 'rb') as file:
    # Load the data from the file
    loaded_data = pickle.load(file)
    # loaded_data is a Dataset object from module Data with attributes "images_train, _val, _test" as well as responses

X_train = torch.tensor(loaded_data.images_train).to(device, dtype=TORCH_DTYPE) #shape (2910,108,108,1) where 108 is the number of pixels. 2910 is the amount of training points
X_val   = torch.tensor(loaded_data.images_val).to(device, dtype=TORCH_DTYPE)
X_test  = torch.tensor(loaded_data.images_test).to(device, dtype=TORCH_DTYPE) # shape (30,108,108,1) # nimages, npx, npx

R_train = torch.tensor(loaded_data.responses_train).to(device, dtype=TORCH_DTYPE) #shape (2910,41) 2910 is the amount of training data, 41 is the number of cells
R_val   = torch.tensor(loaded_data.responses_val).to(device, dtype=TORCH_DTYPE)
R_test  = torch.tensor(loaded_data.responses_test).to(device, dtype=TORCH_DTYPE) # shape (30,30,42) 30 repetitions, 30 images, 42 cells

# Create the complete dataset
X = torch.cat( (X_train, X_val), axis=0,) #shape (3160,108,108,1)
R = torch.cat( (R_train, R_val), axis=0,)

n_px_side = X.shape[1]  

# Reshape images to 1D vector and choose a cell
X = torch.reshape(X, ( X.shape[0], X.shape[1]*X.shape[2])) 
R = R[:,cellid] # shape (nt,) where nt is the number of trials

# Choose a random subset of the data and save the idx
all_idx  = torch.arange(0, X.shape[0])                     # Indices of the whole dataset  
torch.manual_seed(0)
torch.cuda.manual_seed(0)
all_idx_perm  = torch.randperm(all_idx.shape[0])                         # Random permutation of the indices

test_1000_idx = all_idx_perm[-n_test_lk:]                                # Take the last 1000 images out of the set
all_idx_perm  = all_idx_perm[~torch.isin( all_idx_perm, test_1000_idx )] # Remove the test set indices from the permutation

X_test_1000   = X[test_1000_idx,:]
R_test_1000   = R[test_1000_idx]

# endregion

bug_model = utils.load_model(f'../models/bugs/cell:8_negative_variance')

# region __________ Retrieve the values from the last model fit __________
in_use_idx    = bug_model['fit_parameters']['in_use_idx']
xtilde_idx    = bug_model['fit_parameters']['xtilde_idx']

print(xtilde_idx)

remaining_idx = all_idx_perm[~torch.isin( all_idx_perm, in_use_idx )]

X_remaining = X[remaining_idx]
R_remaining = R[remaining_idx]

xtilde = X[xtilde_idx]  # This has to be the same as start_model['xtilde']

xstar  = X_remaining

kernfun       = bug_model['fit_parameters']['kernfun']
if kernfun == 'acosker': kernfun = utils.acosker
n_px_side     = bug_model['fit_parameters']['n_px_side']
EIGVAL_TOL    = bug_model['fit_parameters']['eigval_tol']

final_kernel  = bug_model['final_kernel']

mask          = bug_model['mask']
C             = bug_model['C']
B             = bug_model['B']
K_tilde_b     = bug_model['K_tilde_b']
K_tilde_inv_b = bug_model['K_tilde_inv_b']
K_b           = bug_model['K_b']
Kvec          = bug_model['Kvec']
m_b           = bug_model['m_b']
V_b           = bug_model['V_b']    
f_params      = bug_model['f_params']
theta         = bug_model['hyperparams_tuple'][0]
A             = torch.exp(f_params['logA'])
lambda0       = torch.exp(f_params['loglambda0']) if 'loglambda0' in f_params else f_params['lambda0']

theta_lower_lims  = bug_model['hyperparams_tuple'][1]
theta_higher_lims = bug_model['hyperparams_tuple'][2]

# endregion

# region __________ Calculate the loglikelihood on the 1000 Test set __________

Kvec_test = utils.acosker(theta, X_test_1000[:,mask], x2=None, C=C, dC=None, diag=True)
K_test    = utils.acosker(theta, X_test_1000[:,mask], x2=xtilde[:,mask], C=C, dC=None, diag=False)
K_test_b  = K_test @ B 

lambda_m_t, lambda_var_t = utils.lambda_moments(X_test_1000[:,mask], K_tilde_b, K_test_b@K_tilde_inv_b, Kvec_test, K_test_b, C, m_b, V_b, theta)

f_mean = utils.mean_f_given_lambda_moments(f_params, lambda_m_t, lambda_var_t)

loglk_test_1000_random = utils.compute_loglikelihood(R_test_1000, f_mean, lambda_m_t, lambda_var_t, f_params)[0] 

# endregion    

# region __________ Calculate the utility of each remaining image __________
# Calculate the matrices to compute the lambda moments. They are referred to the unseen images xstar

# xstar = xstar[:1080,:]

Kvec_star = utils.acosker(theta, xstar[:,mask], x2=None, C=C, dC=None, diag=True)
K_star    = utils.acosker(theta, xstar[:,mask], x2=xtilde[:,mask], C=C, dC=None, diag=False)
K_star_b  = K_star @ B 

lambda_m_t, lambda_var_t = utils.lambda_moments( xstar[:,mask], K_tilde_b, K_star_b@K_tilde_inv_b, Kvec_star, K_star_b, C, m_b, V_b, theta)  

logf_mean = A*lambda_m_t + lambda0
logf_var  = A**2 * lambda_var_t

# Estimate the utility and cap the maximum r ( used in a summation to infinity )
r_masked = torch.arange(0, 100, dtype=TORCH_DTYPE)
u2d      = utils.nd_utility(logf_var, logf_mean, r_masked )

# i_best   = u2d.argmax()                 # Index of the best image in the utility vector
i_best = torch.randint(0, u2d.shape[0], (1,)).item()
x_idx_best   = remaining_idx[i_best]    # Index of the best image in the dataset indices
print(f'Utility: {u2d[i_best].item():<8.6f} |  Best image ID: {i_best}  | Best image index: {x_idx_best}')

# endregion

X_remaining.mean().item()


print()





tensor([2753, 1430, 1101,  ...,  341,   27,  647], device='cuda:0')


ValueError: Negative or zero input to log detected